In [1]:
#logging into Rossmanns website with Selenium
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

username = ## enter you username here
password = ## enter your password here

url = "https://www.rossmann.pl/logowanie"

service = webdriver.ChromeService(executable_path = #path to your chromedriver)
driver = webdriver.Chrome(service=service)

driver.get(url)

sleep(3)
driver.find_element(By.ID,"onetrust-accept-btn-handler").click()
driver.find_element(By.ID,"login-user").send_keys(username)
driver.find_element(By.ID,"login-password").send_keys(password)
driver.find_element(By.CLASS_NAME,'ladda-label').click()



In [25]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

def scrape_rossmann(url, gender):
    webpage_html = requests.get(url + f'?DynamicTags=dla%20kogo%5E{gender}')
    soup = BeautifulSoup(webpage_html.text, 'html.parser')
    last_page = int(soup.find('a', class_='pages__last').get_text(strip=True))
    
    products = []
    
    #generate urls for every page
    websites_list = [url + f'?DynamicTags=dla%20kogo%5E{gender}&Page=' + str(page) for page in range(1, last_page + 1)]

    #iterate thru each generated url
    for page_url in websites_list:
        webpage_html = requests.get(page_url)
        soup = BeautifulSoup(webpage_html.text, 'html.parser')
        product_category = re.search(r'kategoria/(.*?),', page_url).group(1)

        #extract product info
        product_tiles = soup.find_all('div', class_='tile-product')
        for product in product_tiles:
            product_name_elem = product.find('a', class_='tile-product__name')
            product_price_elem = product.find('span', class_='tile-product__current-price')
            product_sub_category_elem = product.find('span', class_ = 'Caption-module_infoTextContainer--LSYAe')
            
            if product_name_elem and product_price_elem and product_sub_category_elem:
                product_name = product_name_elem.get_text(strip=True)
                product_price = product_price_elem.get_text(strip=True)
                product_sub_category = product_sub_category_elem.get_text(strip=True)
                
                products.append({
                    'Product Name': product_name,
                    'Price': product_price,
                    'Gender': 'male' if gender == 'mężczyzna' else 'female',
                    'Product Category': product_category,
                    'Product Sub Category and Size': product_sub_category
                })
            else:
                continue

    return products

url_df = pd.read_csv('urls.csv')
url_list = url_df.iloc[:, 0].tolist()

#list to store final product data
all_products = []

# iterate thru each URL
for url in url_list:
    male_products = scrape_rossmann(url, 'mężczyzna')
    female_products = scrape_rossmann(url, 'kobieta')
    
    all_products.extend(male_products)
    all_products.extend(female_products)

df = pd.DataFrame(all_products)
df.to_csv('output2.csv')


In [5]:
##Data cleaning:
##getting rid of the redundant info from the product name column
import pandas as pd
import re

df = pd.read_csv('output2.csv')
df['Product Name'] = df.apply(lambda row: row['Product Name'].replace(row['Product Sub Category and Size'], ''), axis=1)

## splitting the product category column into an additional size column
def extract_size(description):
    import re
    match = re.search(r'(\d+(?:[.,]\d+)?)\s*(ml|g|l|kg|szt.)\b', description, re.IGNORECASE)
    if match:
        return match.group(0)  
    return None

## creating a new column for size
df['Product size'] = df['Product Sub Category and Size'].apply(extract_size)

## removing the product size from the description and also the trailing comma
df['Product Sub Category and Size'] = df['Product Sub Category and Size'].str.replace(r'(\d+(?:[.,]\d+)?)\s*(ml|g|l|kg|oz|lb)\b', '', regex=True, flags=re.IGNORECASE)
df['Product Sub Category and Size'] = df['Product Sub Category and Size'].str.rstrip(',')

## renaming the Product Sub Category and Size column
df = df.rename(columns = {'Product Sub Category and Size':'Product Sub Category'})

##
df['Price'] = df['Price'].str.strip()  ##removing spaces
df['Price'] = df['Price'].str.replace(r'\s*zł$', '', regex=True)  ## removing zloty 
df['Price'] = df['Price'].str.replace(',', '.')  # replacing comma with a dot
df.drop(df.columns[0], axis=1,inplace=True)

df.to_csv('cleaned_data.csv')